In [209]:
import numpy as np

import glob
import re
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats.stats as stats

#from sklearn.decomposition import PCA

In [210]:
import zipfile

# read the dataset using the compression zip
all_data = pd.read_csv('all_data.zip',compression='zip')

In [211]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
0,12027_rev1,NaN,NaN,NaN,NaN,NaN,ABANDONED,Emerson Nolan,1,http://review.couchbase.org/#/c/12027/1
1,12050_rev1,NaN,NaN,NaN,NaN,NaN,MERGED,Emerson Nolan,1,http://review.couchbase.org/#/c/12050/1
2,12050_rev2,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
425910,85948_rev2,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,MERGED,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
425911,85957_rev1,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425912,85957_rev1,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425913,85957_rev1,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


Convert Status to 0 if abandoned and 1 if merged

In [212]:
all_data['status'] = np.where((all_data['status'] == 'MERGED') ,True,False)*1

In [213]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
0,12027_rev1,NaN,NaN,NaN,NaN,NaN,0,Emerson Nolan,1,http://review.couchbase.org/#/c/12027/1
1,12050_rev1,NaN,NaN,NaN,NaN,NaN,1,Emerson Nolan,1,http://review.couchbase.org/#/c/12050/1
2,12050_rev2,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
425910,85948_rev2,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
425911,85957_rev1,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425912,85957_rev1,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425913,85957_rev1,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


In [214]:
all_data['PR_number'].isna().sum()

0

In [215]:
all_data.isna().sum()

PR_number               0
Comment             18374
Author              16440
Date                16377
Commit_message     164415
Descriptions        13489
status                  0
main_author             5
revision_number         0
url                     0
dtype: int64

Drop all rows with nan comments

In [216]:
all_data = all_data.dropna(subset=['Comment'])

In [217]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
2,12050_rev2,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
5,12050_rev2,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
6,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
425910,85948_rev2,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
425911,85957_rev1,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425912,85957_rev1,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
425913,85957_rev1,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


Add a column with only the Pr-number without the patch version

In [218]:
all_data['PR_number_no_patch_number'] = all_data['PR_number'].str[:5]

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\1524495217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['PR_number_no_patch_number'] = all_data['PR_number'].str[:5]


In [219]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
2,12050_rev2,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
3,12050_rev2,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
425910,85948_rev2,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2,85948
425911,85957_rev1,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957
425912,85957_rev1,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957
425913,85957_rev1,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957


Deleting all the comments from bots.

In [220]:
bot_comments = ['Uploaded patch set','Added to reviewer','Verified','Change has been successfully','Your change could not be merged due to a path conflict.','Code-Review','(1 comment)']
bot_comments_pattern = '|'.join(bot_comments)


In [221]:
all_data.Comment.str.contains(bot_comments_pattern).sum()

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\1003850507.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data.Comment.str.contains(bot_comments_pattern).sum()


268554

We have 228249 comments from bots

And now we are going to delete them.

In [222]:
all_data = all_data[all_data.Comment.str.contains(bot_comments_pattern) == False]

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\3367746845.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data = all_data[all_data.Comment.str.contains(bot_comments_pattern) == False]


In [223]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
7,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
8,12050_rev2,Patch Set 2: No scoreThe only reason I brought...,Kenny Maynard,2012-01-05 18:45:06.860000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
425869,85893_rev2,Patch Set 2: Patch Set 1 was rebased,Lucille Zavala,2017-11-23 08:17:17.613000000,Added a method in the rest client to check if ...,"MB-26900: tests should use ""clusterCompatibili...",1,Zaiden Stark,2,http://review.couchbase.org/#/c/85893/2,85893
425883,85919_rev2,Patch Set 2: Patch Set 1 was rebased,Angelina Mayer,2017-11-22 22:29:09.580000000,NaN,CBQE-4378 Fix wrong port extraction and create...,1,Angelina Mayer,2,http://review.couchbase.org/#/c/85919/2,85919
425890,85932_rev1,Patch Set 1:Timothy Maldonados-MBP:testrunner ...,Coleman Colon,2017-11-23 05:36:14.531000000,NaN,CBQE-4391,0,Coleman Colon,1,http://review.couchbase.org/#/c/85932/1,85932
425891,85932_rev1,Patch Set 1:Timothy Maldonados-MBP:testrunner ...,Coleman Colon,2017-11-23 05:36:14.531000000,NaN,CBQE-4391,0,Coleman Colon,1,http://review.couchbase.org/#/c/85932/1,85932


I noticed that only the comments that start with 'Patch Set {number of the patch} are useful for our analysis so I am going to keep only those comments.

In [224]:
useful_data = all_data[all_data['Comment'].str[:9].str.contains('Patch Set') == True]

In [225]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
4,12050_rev2,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
7,12050_rev2,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
8,12050_rev2,Patch Set 2: No scoreThe only reason I brought...,Kenny Maynard,2012-01-05 18:45:06.860000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
425743,85636_rev2,Patch Set 2: Patch Set 1 was rebased,Emely Wilson,2017-11-16 09:04:15.878000000,NaN,CBQE-4333: Fix aggregate event processing stat...,1,Emely Wilson,2,http://review.couchbase.org/#/c/85636/2,85636
425869,85893_rev2,Patch Set 2: Patch Set 1 was rebased,Lucille Zavala,2017-11-23 08:17:17.613000000,Added a method in the rest client to check if ...,"MB-26900: tests should use ""clusterCompatibili...",1,Zaiden Stark,2,http://review.couchbase.org/#/c/85893/2,85893
425883,85919_rev2,Patch Set 2: Patch Set 1 was rebased,Angelina Mayer,2017-11-22 22:29:09.580000000,NaN,CBQE-4378 Fix wrong port extraction and create...,1,Angelina Mayer,2,http://review.couchbase.org/#/c/85919/2,85919
425890,85932_rev1,Patch Set 1:Timothy Maldonados-MBP:testrunner ...,Coleman Colon,2017-11-23 05:36:14.531000000,NaN,CBQE-4391,0,Coleman Colon,1,http://review.couchbase.org/#/c/85932/1,85932


Now we are going to remove the Patch set number from every comment.

In [226]:
useful_data['Comment'] = all_data['Comment'].str[12:]

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\13972313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Comment'] = all_data['Comment'].str[12:]


Find the length of every comment

In [227]:
useful_data['comment_length'] = useful_data['Comment'].str.len()

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\2463225354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['comment_length'] = useful_data['Comment'].str.len()


Removing all the comments with more than 500 characters.


In [228]:
useful_data = useful_data[useful_data['comment_length'] <= 500]

In [229]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
425699,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75
425737,85613_rev1,"Need to verify it on both, centos and windows",Frederick Booker,2017-11-17 18:42:57.845000000,NaN,CBQE-4378 String manipulation in bash incorrec...,1,Angelina Mayer,1,http://review.couchbase.org/#/c/85613/1,85613,45
425743,85636_rev2,Patch Set 1 was rebased,Emely Wilson,2017-11-16 09:04:15.878000000,NaN,CBQE-4333: Fix aggregate event processing stat...,1,Emely Wilson,2,http://review.couchbase.org/#/c/85636/2,85636,24
425869,85893_rev2,Patch Set 1 was rebased,Lucille Zavala,2017-11-23 08:17:17.613000000,Added a method in the rest client to check if ...,"MB-26900: tests should use ""clusterCompatibili...",1,Zaiden Stark,2,http://review.couchbase.org/#/c/85893/2,85893,24


In [230]:
useful_data['comment_length'] = useful_data['Comment'].str.len()

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\2463225354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['comment_length'] = useful_data['Comment'].str.len()


In [232]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
425699,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75
425737,85613_rev1,"Need to verify it on both, centos and windows",Frederick Booker,2017-11-17 18:42:57.845000000,NaN,CBQE-4378 String manipulation in bash incorrec...,1,Angelina Mayer,1,http://review.couchbase.org/#/c/85613/1,85613,45
425743,85636_rev2,Patch Set 1 was rebased,Emely Wilson,2017-11-16 09:04:15.878000000,NaN,CBQE-4333: Fix aggregate event processing stat...,1,Emely Wilson,2,http://review.couchbase.org/#/c/85636/2,85636,24
425869,85893_rev2,Patch Set 1 was rebased,Lucille Zavala,2017-11-23 08:17:17.613000000,Added a method in the rest client to check if ...,"MB-26900: tests should use ""clusterCompatibili...",1,Zaiden Stark,2,http://review.couchbase.org/#/c/85893/2,85893,24


Deleting again some bots comments.

In [238]:
bot_comments = ['Patch Set','Cherry Picked from branch master.','cherry picked','Cherry Picked']
bot_comments_pattern = '|'.join(bot_comments)
useful_data = useful_data[useful_data.Comment.str.contains(bot_comments_pattern) == False]

In [239]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
425032,84162_rev1,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61
425043,84212_rev1,Logs : https://gist.github.com/Emely Wilson/17...,Emely Wilson,2017-10-10 11:17:15.392000000,NaN,CBQE-4333: Refactor eventing code,1,Emely Wilson,1,http://review.couchbase.org/#/c/84212/1,84212,76
425098,84284_rev1,Cherry Picked from branch watson.,Monique Barker,2017-10-11 18:40:50.760000000,NaN,Test for MB-25737:group by on empty table shou...,1,Dylan Paul,1,http://review.couchbase.org/#/c/84284/1,84284,34
425699,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75


The Flesch-Kincaid Grade Level

In [240]:
len(useful_data)

109779

In [241]:
useful_data['Flesch-Kincaid'] = 0

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\436516747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Flesch-Kincaid'] = 0


In [242]:
import textstat

for n in range(len(useful_data)):
    useful_data['Flesch-Kincaid'].iloc[n] = textstat.flesch_reading_ease(useful_data['Comment'].iloc[n])
#textstat.flesch_reading_ease(text)

C:\Users\GNR\AppData\Local\Temp\ipykernel_12628\1877658995.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Flesch-Kincaid'].iloc[n] = textstat.flesch_reading_ease(useful_data['Comment'].iloc[n])


In [245]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid
4,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27
5,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62
9,12050_rev2,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74
10,12050_rev2,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33
12,12050_rev3,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425032,84162_rev1,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77
425043,84212_rev1,Logs : https://gist.github.com/Emely Wilson/17...,Emely Wilson,2017-10-10 11:17:15.392000000,NaN,CBQE-4333: Refactor eventing code,1,Emely Wilson,1,http://review.couchbase.org/#/c/84212/1,84212,76,9.21
425098,84284_rev1,Cherry Picked from branch watson.,Monique Barker,2017-10-11 18:40:50.760000000,NaN,Test for MB-25737:group by on empty table shou...,1,Dylan Paul,1,http://review.couchbase.org/#/c/84284/1,84284,34,83.32
425699,85583_rev2,log : https://gist.github.com/Emely Wilson/8f5...,Emely Wilson,2017-11-15 12:05:39.021000000,NaN,CBQE-4333: Add eventing settings tests,1,Emely Wilson,2,http://review.couchbase.org/#/c/85583/2,85583,75,-50.02


In [244]:
compression_opts = dict(method='zip',
                        archive_name='all_data_for_analysis.csv') 
useful_data.to_csv('all_data_for_analysis.zip', index=False, compression=compression_opts)

In [ ]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,Flesch-Kincaid
4,12050_rev2,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,29.01
5,12050_rev2,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,29.01
6,12050_rev2,I don't know that changing the package name is...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,29.01
7,12050_rev2,I don't know that changing the package name is...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,29.01
8,12050_rev2,No scoreThe only reason I brought it up was t...,Kenny Maynard,2012-01-05 18:45:06.860000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,29.01
...,...,...,...,...,...,...,...,...,...,...,...,...
425896,85933_rev1,Code-Review+2,Emely Wilson,2017-11-23 06:34:46.137000000,NaN,CBQE-4391,0,Coleman Colon,1,http://review.couchbase.org/#/c/85933/1,85933,29.01
425901,85939_rev1,Code-Review+2,Johan Savage,2017-11-23 09:27:07.997000000,NaN,"Test change, do not submit",0,Johan Savage,1,http://review.couchbase.org/#/c/85939/1,85939,29.01
425904,85939_rev1,Code-Review+2,Alissa Huang,2017-11-23 09:43:52.348000000,NaN,"Test change, do not submit",0,Johan Savage,1,http://review.couchbase.org/#/c/85939/1,85939,29.01
425909,85948_rev1,Code-Review+2,Lucille Zavala,2017-11-23 10:50:48.107000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,1,http://review.couchbase.org/#/c/85948/1,85948,29.01


The Fog Scale (Gunning FOG Formula)

In [ ]:
textstat.gunning_fog(text)

0.0

Automated Readability Index

In [ ]:
textstat.automated_readability_index(text)

12.2